<a href="https://colab.research.google.com/github/gomdoori/user1/blob/main/Chatgpt%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##step0) Colab 환경 설정
설치(python>=3.8)

In [ ]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7

# setup data

%cd /content/drive/MyDrive/인공지능강의/GPT/colossalai_ChatGPT/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.7/686.7 kB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 68.4 MB/s eta 0:00:00
     ━━━━━━

##Step 1) SFT: 질문에 대답을 잘하는 모델 만들기
SFT(Supervised Fine Tuning)

*   질문에 응답을 잘하도록 SFT 수행
*   사람이 지시한 대답(13,000개), 질문에 응답(13,000개)
* 질문-응답의 쌍으로 이루어진 데이터 셋






In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn 
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_1_SFT', type=str, default='/content/drive/MyDrive/인공지능강의/GPT/data_kochatgpt/kochatgpt_1_SFT_custom.jsonl')
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default='./output_1_SFT')

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
args.max_epochs = 2
print(args)

Namespace(data_path_1_SFT='/content/drive/MyDrive/인공지능강의/GPT/data_kochatgpt/kochatgpt_1_SFT_custom.jsonl', model_name='skt/kogpt2-base-v2', max_epochs=2, train_batch_size=8, output_dir='./output_1_SFT')


In [ ]:
## test & load skt gpt2 kroean
import torch
from transformers import GPT2LMHeadModel

from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
print(tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o"))

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는 어떻게 해야할까요?'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


근육이 커지기 위해서는 어떻게 해야할까요?"
"그렇다면 그건 바로 '내게 맞는' 방법입니다. 그리고 이 방법은 아주 간단합니다. 우선, 먼저 내 몸에 있는 지방을 제거해 줍니다. 지방세포는 우리 몸에서 가장 많이 분포돼 있습니다. 따라서 지방이 많은 사람은 자신의 몸 속에 들어있는 지방의 양을 줄여야 합니다. 그래서 나는 이것을 통해 체중을 감량하고 싶습니다. 하지만 너무 쉽게 살을 빼는 것은 좋지 않으니 주의해야겠죠. 왜냐하면 지방은 대부분 근육과 인대 등에 분포되어 있기 때문에 그만큼의 양이 많기 때문이지요. 그러므로 다이어트를 할 때는 반드시 전문의와 상의해야 하겠지요?
이렇게 하면 살이 찌기 전에


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
print(len(tokenizer.tokenize("하늘은 파란색 입니다.")))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


6


In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375,
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

generator(
    ["0 : AI에 관심이 있니?\n:",
    "0 : AI라는 것은 너무 재미있는거 같아\n: 맞아 AI는 너무 흥미로워 \n: 너도 그렇게 느끼니? 맞아 AI라는것은 너무 다양하고 신기해\n:"],
    **generation_args
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[[{'generated_text': '0 : AI에 관심이 있니?\n: __아직까지는 __없다\n__;\n'}],
 [{'generated_text': '0 : AI라는 것은 너무 재미있는거 같아\n: 맞아 AI는 너무 흥미로워 \n: 너도 그렇게 느끼니? 맞아 AI라는것은 너무 다양하고 신기해\n: 괜찮아?\n: 좋아, 좋아, 좋아.\n: 너희들끼리 얘기하는게 재미있어\n'}]]

In [ ]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Instruction(명령어):{prompt}\n\n###Input(입력):\n{input}\n\nResponse(응답):"
    ),
    "prompt_no_input": (
        "Instruction(명령어):{prompt}\n\nResponse(응답):"
    ),
}

In [ ]:
## 모델 준비
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,    
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)    
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [ ]:
## prepare data
from typing import Optional, Dict, Sequence
    
class SFT_dataset(Dataset):
    '''SFT dataset by wygo'''
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")
        
        ## format
        pattern_instruction = 'prompt'  # instruction
        pattern_input = 'input'  
        pattern_output = 'completion'  

        ############################################################
        ## load dataset

        data_path_1_SFT = '/content/drive/MyDrive/인공지능강의/GPT/data_kochatgpt/kochatgpt_1_SFT_custom.jsonl'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)
            if verbose:
                print('## data check ##')
                print((list_data_dict[0]))

        ############################################################
        ## 데이터셋 만들기, source와 target
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]  # 템플릿 가져오기

        # 입력
        sources = []
        for example in list_data_dict:
            if example.get(pattern_input, "") != "":
                tmp = prompt_input.format_map(example)
            else:
                tmp = prompt_no_input.format_map(example)
            sources.append(tmp)

        # 출력
        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

        if verbose:
            idx = 0
            print((sources[idx]))
            print((targets[idx]))
            print("Tokenizing inputs... This may take some time...")

        ############################################################
        examples = [s + t for s, t in zip(sources, targets)]

        # source data tokenized
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target


        ## 입력은 source, 출력은 source+target 이지만 학습은 target 부분만
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다
        
        data_dict = dict(input_ids=input_ids, labels=labels)        
        
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))    
        
    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        """Tokenize a list of strings."""
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )        
        
        
    def __len__(self):
        return len(self.input_ids)

    
    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

    

train_dataset = SFT_dataset(data_path_1_SFT=args.data_path_1_SFT, tokenizer=tokenizer)
eval_dataset  = None  # eval은 안함
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

# check
print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([14659, 13394, 37091, 10651,   383, 25841,  8006, 14914,  7673, 20479,
         8091, 22311,  9036, 30902, 13675,   375,   424,  9792,   454,  9549,
        20549,   383,  8142,  7192, 14914,   382, 37767, 13753,  8263,  7166,
          739,  8352,  7659,  9594, 25585, 13600,  8022,  9378, 11532,  9887,
        11218,  9111, 16691, 10351, 10561,  9128, 20479,  8091,  9065,  9446,
         9036, 28420, 26521, 10163, 26367,  6958,  9030,  9882, 12317, 25882,
         9209, 37194, 10351,  9036, 12168, 10529, 15989,  9719, 15434, 10552,
        11188, 13362,  9036, 15805, 11300, 11846,  9146, 16691,  9181,  7397,
        15806, 13480, 11342, 17596,  9161, 19996,  9025, 25006, 18595,  9966,
        12592, 10751, 11814,  8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,   38

In [ ]:
#!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
## 학습 (10min)
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/인공지능강의/GPT", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 3, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved 
    warmup_steps=5,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.113300
1000,2.952300
1500,2.849600
2000,2.786300
2500,2.704300
3000,2.664700


##GPT2모델이 사람의 질문에 대해 대답하는 모델을 학습함

In [ ]:
## 추론 테스트
generator = pipeline('text-generation', model=args.output_dir, tokenizer=tokenizer)
# generator = pipeline('text-generation', model=model.cpu(), tokenizer=tokenizer, config={'max_length':800})

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0, #반복되는 텍스트를 억제함
    no_repeat_ngram_size=4, #개의 연속적인 단어를 반복하지 않는다.
    eos_token_id=375, #EOS(End-of-Sequence)를 나타내는 토큰 ID를 지정 및 텍스트를 생성할 때 모델은 이 EOS 토큰에 도달하면 토큰 생성을 중지
    max_new_tokens=64,#생성하는 토큰의 최대 수
    do_sample=True, #True로 설정하면 모델이 예측 확률에 따라 다음 토큰을 무작위 선택 
    top_k=50, #다음 토큰을 예측할때 예측 확률이 가장 높은 상위 k 개로 예측함
    early_stopping=True
)

list_prompt = ["AI란 무엇인가?", "강원도는 무엇이 유명한가요?"]
list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print(("-----------------Chat AI를 실행합니다.-----------------\n"))
    print(('질문 : {}\n\n').format(result[0]['generated_text']))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


-----------------Chat AI를 실행합니다.-----------------

질문 : Instruction(명령어):AI란 무엇인가?

Response(응답):'AI란 인공지능 어시스턴트이기 때문에 인간과 같은 감정을 느끼지는 않습니다. 하지만 AI는 다양한 분야에서 활용될 수 있습니다. 예를 들어, 로봇, 컴퓨터, 스마트폰, PDF 파일 등 다양한 분야에서 활용됩니다. AI를 통해 인간과는 다른 경험을 할 수 있습니다. AI


-----------------Chat AI를 실행합니다.-----------------

질문 : Instruction(명령어):강원도는 무엇이 유명한가요?

Response(응답):'강원도에는 다양한 관광지가 있습니다. 대표적인 관광지로는 영월, 삼척, 속초, 고성 등이 있습니다. 또한 강릉과 속초도 대표적인 관광지 중 하나입니다. 高敞, 固城, 江原, 永川 등이 있습니다. 高敞은 조선 시대 문신인 노자규(李子圭)가 지은




##Step 2) RM : 좋은 글 채점기 만들기

* 생성된 각 글에 대한 점수를 매기고 이를 보상모델로 만듬

In [ ]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7
!pip install torch==1.13.1
# setup data

%cd /content/drive/MyDrive/인공지능강의/GPT/colossalai_ChatGPT/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you ha

In [ ]:
import argparse

import loralib as lora
import torch
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from datasets import load_dataset
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

import os
import json

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Instruction(명령어):{prompt}\n\n### Input(입력):\n{input}\n\n Response(응답):"
    ),
    "prompt_no_input": (
        "Instruction(명령어):{prompt}\n\n Response(응답):"
    ),
}

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default='./output_2_RM')
parser.add_argument('--data_path_2_RM', type=str, default='/content/drive/MyDrive/인공지능강의/GPT/data_kochatgpt/kochatgpt_2_RM_custom.jsonl')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_len', type=int, default=512)  # wygo 추가

args = parser.parse_args(args=[])

# for test
args.max_epochs = 3
args.pretrain = 'skt/kogpt2-base-v2'  # pretrained 모델 가져오기
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# customizing, https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py#L29
from typing import Optional

import torch.nn as nn
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# from ..base import RewardModel
from chatgpt.models.base import RewardModel


class GPTRM_custom(RewardModel):
    """
    GPT Reward model.
    Args:
        pretrained (str): Pretrained model name or path.
        config (GPT2Config): Model config.
        checkpoint (bool): Enable gradient checkpointing.
        lora_rank (int): Rank of the low-rank approximation.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))  # wygo 추가!!!
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        # model = model.resize_token_embeddings(len(tokenizer))

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    # load pretrained gpt2    
    if args.model == 'gpt2':
#         tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer = AutoTokenizer.from_pretrained(args.pretrain)
        tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )
        tokenizer.pad_token = tokenizer.eos_token
        model = GPTRM_custom(pretrained=args.pretrain, lora_rank=args.lora_rank, tokenizer=tokenizer).cuda()

    elif args.model == 'bloom':
        model = BLOOMRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)
    
    elif args.model == 'opt':
        model = OPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")      
    
    else:
        raise ValueError(f'Unsupported model "{args.model}"')
    
    
    # model.resize_token_embeddings(len(tokenizer))

In [ ]:
# make ranking data to chosen, rejetced data
with open(args.data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    if args.verbose:
        print('## data check ##')
        print((list_data_dict[0]))
        
total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # data 1) 0 VS 1
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)


    # data 2) 0 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # data 1) 1 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)
    
    
    
    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

In [ ]:
len(total_data_ranking2chosen)

In [ ]:
# prepare for data and dataset
import random
random.seed(230319)
# list_tmp = list(range(10))
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

# train_data = total_data_ranking2chosen[:-1000]  # 29000 학습
# eval_data = total_data_ranking2chosen[-1000:0]  # 1000개만 평가

train_data = total_data_ranking2chosen[:100]  # 학습 데이터의 수
eval_data = total_data_ranking2chosen[100:130]  # 평가 데이터의 수

train_dataset = RewardDataset(train_data, tokenizer, args.max_len)
eval_dataset = RewardDataset(eval_data, tokenizer, args.max_len)

# check
idx = 10
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

In [ ]:
# configure optimizer
if args.strategy.startswith('colossalai'):
    optim = HybridAdam(model.parameters(), lr=5e-5)
else:
    optim = Adam(model.parameters(), lr=5e-5)

In [ ]:
# batch_size here is expected to be C(k,2), k means # response of each prompt
# be limited with the format of dataset 'Dahoas/rm-static', we'd better use batch_size as 1
trainer = RewardModelTrainer(model=model,
                             strategy=strategy,
                             optim=optim,
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=args.batch_size,
                             max_epochs=args.max_epochs)

##RM을 통해 각 순위에 대해 점수를 부여하고 가장 정확한 응답에 대해 학습함

In [ ]:
# train!!
trainer.fit(use_lora=args.lora_rank)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(model, os.path.join(args.output_dir, 'RM.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(optim,
                        os.path.join(args.output_dir, 'RM_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

## Step 3) PPO 사람의 피드백을 반영하여 질문에 대한 응답이 피드백과 비슷해 지도록 학습

In [ ]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7
!pip install torch==1.13.1
# setup data

%cd /content/drive/MyDrive/인공지능강의/GPT/colossalai_ChatGPT/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

In [ ]:
# import
import argparse
from copy import deepcopy

import pandas as pd
import torch
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMActor, BLOOMCritic
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.opt import OPTActor, OPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

## wy 추가
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## clossalAI error 해결
os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '42043'

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Instruction(명령어):{prompt}\n\n### Input(입력):\n{input}\n\nResponse(응답):"
    ),
    "prompt_no_input": (
        "Instruction(명령어):{prompt}\n\nResponse(응답):"
    ),
}

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_3_PPO', type=str, default='/content/drive/MyDrive/인공지능강의/GPT/data_kochatgpt/kochatgpt_3_PPO.jsonl')
parser.add_argument('--output_dir', type=str, default='./output_3_PPO')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--num_episodes', type=int, default=10)
parser.add_argument('--max_timesteps', type=int, default=3)
parser.add_argument('--update_timesteps', type=int, default=3)
parser.add_argument('--max_epochs', type=int, default=5)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_length', type=int, default=250)
args = parser.parse_args(args=[])

# for test
args.output_dir = './output_3_PPO'
args.pretrain = 'skt/kogpt2-base-v2'  # pretrained 모델 가져오기

# args.pretrain_actor = './output_1_SFT'  # SFT 모델 가져오기
# args.pretrain_critic = './output_2_RM'  # RM 모델 가져오기
args.pretrain_actor = args.pretrain
args.pretrain_critic = args.pretrain

args.num_episodes = 1
args.max_epochs   = 1

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
args.pretrain

In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    if args.model == 'gpt2':
        actor = GPTActor(pretrained=args.pretrain_actor, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        critic = GPTCritic(pretrained=args.pretrain_critic, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer.pad_token = tokenizer.eos_token
        tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )    
        tokenizer.pad_token = tokenizer.eos_token



    elif args.model == 'bloom':
        actor = BLOOMActor(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        critic = BLOOMCritic(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)
        tokenizer.pad_token = tokenizer.eos_token            
    elif args.model == 'opt':
        actor = OPTActor(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        critic = OPTCritic(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")            
    else:
        raise ValueError(f'Unsupported model "{args.model}"')

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [ ]:
# configure optimizer
if args.strategy.startswith('colossalai'):
    actor_optim = HybridAdam(actor.parameters(), lr=5e-6)
    critic_optim = HybridAdam(critic.parameters(), lr=5e-6)
else:
    actor_optim = Adam(actor.parameters(), lr=5e-6)
    critic_optim = Adam(critic.parameters(), lr=5e-6)

In [ ]:
# setting the models
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = strategy.prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [ ]:
# prepare data
with open(args.data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(list_prompt)
print('\n\n\n')
print(tokenize_fn('I want you to act as a linux terminal.'))

In [ ]:
# configure trainer
trainer = PPOTrainer(strategy,
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=args.max_epochs,
                     train_batch_size=args.train_batch_size,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

## train!
trainer.fit(list_prompt,  # 입력 prompt
            num_episodes=args.num_episodes,
            max_timesteps=args.max_timesteps,
            update_timesteps=args.update_timesteps)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(actor, os.path.join(args.output_dir, 'actor.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(actor_optim,
                        os.path.join(args.output_dir, 'actor_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

In [ ]:
actor=GPTActor(pretrained=args.pretrain_actor, lora_rank=args.lora_rank)

In [ ]:
## inference
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=args.max_length,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print("-"*70)
    print(("챗봇 : {}").format(output))
    print("-"*70)
    return output

In [ ]:
list_prompt = ["인공지능은 무엇인가 설명해줘","하늘에는 왜 구름이 있을까?"]

for input_text in list_prompt:
  print("-"*70)
  print(("질문 : {}").format(input_text))
  output = generation(input_text)

##학습 데이터 활용하기

In [ ]:
# import
import argparse

import torch
torch.cuda.empty_cache()
from chatgpt.models.bloom import BLOOMActor
from chatgpt.models.gpt import GPTActor
from chatgpt.models.opt import OPTActor
from transformers import AutoTokenizer
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Instruction(명령어):{prompt}\n\n### Input(입력):\n{input}\n\nResponse(응답):"
    ),
    "prompt_no_input": (
        "Instruction(명령어):{prompt}\n\nResponse(응답):"
    ),
}

In [ ]:
import os
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--model',
                    default='gpt2',
                    choices=['gpt2', 'bloom', 'opt'])
# We suggest to use the pretrained model from HuggingFace, use pretrain to configure model
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--model_path', type=str, default=None)
parser.add_argument('--input',
                    type=str,
                    default='Question: How are you ? Answer:')
parser.add_argument('--max_length', type=int, default=250)
args_inference = parser.parse_args([])

args_inference.model = 'gpt2'
args_inference.pretrain = 'skt/kogpt2-base-v2'
args_inference.model_directory = '/content/drive/MyDrive/인공지능강의/output_3_PPO'
args_inference.model_path = os.path.join(args_inference.model_directory, 'actor.pt')

# configure model, tokenizer
if args_inference.model == 'gpt2':
    #actor = GPTActor(pretrained=args_inference.pretrain).to(torch.cuda.current_device())
    actor = GPTActor(pretrained=args_inference.pretrain)
    # tokenizer = GPT2Tokenizer.from_pretrained(args_inference.pretrain)
    # tokenizer.pad_token = tokenizer.eos_token
    tokenizer = AutoTokenizer.from_pretrained(args_inference.pretrain,
                                              padding_side="right",
                                              model_max_length=512)
    tokenizer.add_special_tokens({
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    })
    tokenizer.pad_token = tokenizer.eos_token

elif args_inference.model == 'bloom':
    actor = BLOOMActor(pretrained=args_inference.pretrain).to(
        torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-560m')
    tokenizer.pad_token = tokenizer.eos_token
elif args_inference.model == 'opt':
    actor = OPTActor(pretrained=args_inference.pretrain).to(torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained('facebook/opt-350m')
else:
    raise ValueError(f'Unsupported model "{args_inference.model}"')

state_dict = torch.load(args_inference.model_path, map_location='cpu');
actor.model.load_state_dict(state_dict);

actor.eval();

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
## inference
def generation(input_text):
    #input_ids = tokenizer.encode(input_text, return_tensors='pt').to(torch.cuda.current_device())
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    outputs = actor.generate(input_ids,
                             max_length=args_inference.max_length,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print("-"*70)
    print(("챗봇 : {}").format(output))
    print("-"*70)
    return output

In [ ]:
list_prompt = ["인공지능은 무엇인가 설명해줘","하늘에는 왜 구름이 있을까?"]

for input_text in list_prompt:
  print("-"*70)
  print(("질문 : {}").format(input_text))
  output = generation(input_text)

----------------------------------------------------------------------
질문 : 인공지능은 무엇인가 설명해줘
----------------------------------------------------------------------
챗봇 : 인공지능은 무엇인가 설명해줘요.
어머님이 보시는 게 바로 그거예요?
어머님은 그게 뭔지 아세요?
아니 그냥 그~ 알 수가 없는 건데 왜 그~ 그~ 그~ 봇물 터지는 그~ 영상을 보고 진짜 어머님은 얼마나 이쁘게 보셨을까라고 생각하시겠지만 그~ 그~ 그~ 그~ 어머님이 이게 무슨 소릴 하시는지 몰랐지만 이제 그~ 뭐랄까 되게 궁금하잖아요.
그~ 그~ 그~ 아~ 그~ 그~ 그~ 그~ 니까?
아~ 그래서 그~ 아~ 제가 그~ 지금 그~ 니까?
그래서 그~ 그~ 니까?
그~ 아~ 아니 근데 아니 그~ 니까?
그~ 그~ 아~ 그럼 어~ 뭐야 아~ 그~ 아~ 어~ 이~ 그~ 뭔지 궁금하잖아요.
그~ 니?
그~ 그~ 니까?
그~ 아~ 뭔지 궁금한 거예요?
아니 니까?
니까?
아니 뭐야 그래서 니까?
아니 그~ 저희 아~ 그~
----------------------------------------------------------------------
----------------------------------------------------------------------
질문 : 하늘에는 왜 구름이 있을까?
----------------------------------------------------------------------
챗봇 : 하늘에는 왜 구름이 있을까? 어때? 지금이 땡땡이이야. 우리 집이 다 좋았지."
그 말에 놀란 엄마는 말했다.
"나는 다들 좋았어. 난 엄마한테 잘됐어. 너희들 다 잘됐어. 내 아들도 엄마가 잘됐으면 좋겠다고. 우리 집 사람들이 잘돼서 좋았어."
"그렇다면 이제 집으로 돌아가도 돼?"
"내가 엄마한테 잘됐으면 좋겠어."
"그럼, 우리 집